In [1]:
# Install necessary libraries
!pip install tensorflow numpy matplotlib kagglehub

# Import dependencies
import kagglehub
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import os

# Verify if the dataset path exists
dataset_path = '/Users/parthsoni/.cache/kagglehub/datasets/danushkumarv/indian-monuments-image-dataset/versions/1/validation/Indian-monuments/images/train'

if os.path.exists(dataset_path):
    print("✅ Dataset path exists!")
else:
    print("❌ Dataset path NOT found!")

✅ Dataset path exists!


In [3]:
# Dataset Path
dataset_path = '/Users/parthsoni/.cache/kagglehub/datasets/danushkumarv/indian-monuments-image-dataset/versions/1/validation/Indian-monuments/images/train'

# Define Image Parameters
img_size = (224, 224)
batch_size = 32

# Data Augmentation & Splitting
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.3,  # Increased validation split to prevent overfitting
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Generators
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Print class mapping
print("Class Mapping:", train_generator.class_indices)

Found 2578 images belonging to 24 classes.
Found 1088 images belonging to 24 classes.
Class Mapping: {'Ajanta Caves': 0, 'Charar-E- Sharif': 1, 'Chhota_Imambara': 2, 'Ellora Caves': 3, 'Fatehpur Sikri': 4, 'Gateway of India': 5, 'Humayun_s Tomb': 6, 'India gate pics': 7, 'Khajuraho': 8, 'Sun Temple Konark': 9, 'alai_darwaza': 10, 'alai_minar': 11, 'basilica_of_bom_jesus': 12, 'charminar': 13, 'golden temple': 14, 'hawa mahal pics': 15, 'iron_pillar': 16, 'jamali_kamali_tomb': 17, 'lotus_temple': 18, 'mysore_palace': 19, 'qutub_minar': 20, 'tajmahal': 21, 'tanjavur temple': 22, 'victoria memorial': 23}


In [4]:
# Load Pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base layers

# Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)  # Added dropout to prevent overfitting
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)  
outputs = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Compile Model
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Use Early Stopping to prevent overfitting

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train with more epochs and callbacks
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,  # Increase epochs
    callbacks=[early_stopping, reduce_lr]
)

# Save the model
model.save("monument_resnet_model.h5")
print("✅ Model saved successfully.")

Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1279 - loss: 2.9377  